In [21]:
import signal
import requests
import time
import sys
import ritc
import pandas as pd

In [2]:
API_KEY = {'X-API-Key':'3PAFQ3IA'}

In [50]:
def get_ticker(df1 = []):
    payload = {}
    headers = {
    'X-API-Key': '3PAFQ3IA',
    'ticker':'RB'
    }
    try:
        current_tick = requests.get(f"http://localhost:9999/v1/case", headers=headers, data=payload)
        current_tick = current_tick.json()['tick']
        securities = requests.get(f"http://localhost:9999/v1/securities", headers=headers, data=payload)
        securities = securities.json()
        news = requests.get(f"http://localhost:9999/v1/news", headers=headers, data=payload)
        news = news.json()[0]['body']
        df_securities = pd.DataFrame(securities)
        df_securities['current_tick'] = current_tick
        df_securities['lastest_news'] = news
        if len(df1)>0:
            joined_dataframe = pd.concat([df1, df_securities], axis=0, ignore_index=True)        
            return joined_dataframe
    except:
        print("Case not active or error while getting data")    
    return df_securities

In [52]:
def main_price_check():
    Flag = True
    try:
        csv_file_path = 'Price.csv'
        df_securities = pd.read_csv(csv_file_path)
    except:
        df_securities = []    
    payload = {}
    headers = {
    'X-API-Key': '3PAFQ3IA',
    'ticker':'RB'
    }
    while Flag:
        status = requests.get(f"http://localhost:9999/v1/case", headers=headers, data=payload)
        status = status.json()['status']
        if status == 'ACTIVE':
            if len(df_securities) == 0:
                df_securities = get_ticker()
            else:
                df_securities = get_ticker(df_securities)
            time.sleep(0.994)
        else:
            time.sleep(1)
            Flag =False          
    df_securities.to_csv(f'Price.csv', index=False)

In [53]:
def main():
    my_counter = 12
    for a in range(my_counter):
        main_price_check()
        time.sleep(61)
    print(f"Done with {my_counter} rounds")

In [54]:
main()

IndexError: list index out of range